In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import matplotlib.pyplot as plt

from pyciemss.PetriNetODE.interfaces import (setup_petri_model,intervene_petri_model,
                                             sample_petri, calibrate_petri)

from pyciemss.utils import get_tspan
from pyciemss.utils.interface_utils import convert_to_output_format
from pyciemss.visuals import plots
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


# Deterministic Lotka Volterra

## Load model

In [5]:
from pyciemss.PetriNetODE.models import LotkaVolterra

raw_deterministic_lotka_volterra = LotkaVolterra(alpha=.67, beta=1.33, gamma=1.00, delta=1.00, add_uncertainty=False)
raw_deterministic_lotka_volterra

LotkaVolterra()

## Initialize model

In [6]:
deterministic_lotka_volterra = setup_petri_model(raw_deterministic_lotka_volterra, start_time=0.0, start_state=dict(prey_population=1.0, predator_population=1.0))
deterministic_lotka_volterra

LotkaVolterra()

In [7]:
tspan = get_tspan(1, 50, 500)

## Sample trajectory

In [8]:
deterministic_lotka_volterra_trajectory = sample_petri(deterministic_lotka_volterra, tspan, 1)
deterministic_lotka_volterra_trajectory['prey_population_sol'][0][0]

tensor(0.5775)

In [11]:
deterministic_lotka_volterra_trajectory_df = convert_to_output_format(deterministic_lotka_volterra_trajectory,
                                                                      tspan, time_unit="years")
schema = plots.trajectories(deterministic_lotka_volterra_trajectory_df)
plots.ipy_display(schema)


# Load lotka volterra  with uncertainty

In [12]:
from pyciemss.PetriNetODE.models import LotkaVolterra

uncertain_lotka_volterra_raw= LotkaVolterra(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, pseudocount=10)
uncertain_lotka_volterra_raw

LotkaVolterra()

## Initialize uncertain Lotka-volterra model

In [13]:
uncertain_lotka_volterra = setup_petri_model(uncertain_lotka_volterra_raw,
                                             start_time=0.0, 
                                             start_state=dict(
                                                 prey_population=1.00, 
                                                 predator_population=1.00))
uncertain_lotka_volterra

LotkaVolterra()

## Sample from prior

In [14]:
prior_samples = sample_petri(uncertain_lotka_volterra, tspan, 50)
prior_samples.keys()

dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [15]:
prior_samples_df = convert_to_output_format(prior_samples,
                                            tspan, time_unit="years")
schema = plots.trajectories(prior_samples_df, 
                           keep=".*_sol",
                           relabel={"prey_population_sol": "prey_prior",
                                    "predator_population_sol": "predator_prior"})
plots.ipy_display(schema)

## Calibrate uncertain Lotka Volterra

We make one observation per day. Because we simulate 500 data points over 50 days, we need to sample every 10th timepoint.

In [16]:
fitted_parameters = calibrate_petri(uncertain_lotka_volterra, 
                                data =[(i, dict(prey_population=deterministic_lotka_volterra_trajectory['prey_population_sol'][0][i*10]))
                                       for i in range(1,50)])
fitted_parameters

AutoLowRankMultivariateNormal()

In [17]:
posterior_samples = sample_petri(uncertain_lotka_volterra, 
                                 timepoints=tspan, num_samples=50, 
                           inferred_parameters=fitted_parameters)

posterior_samples.keys()


dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [28]:
posterior_samples_df = convert_to_output_format(posterior_samples,
                                                tspan, time_unit="years")

distributions = pd.DataFrame({
    'timepoint_years': prior_samples_df['timepoint_years'],
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_posterior': posterior_samples_df['prey_population_sol'],
    'predator_posterior': posterior_samples_df['predator_population_sol'],
    })

traces = pd.DataFrame({
    'timepoint_years': deterministic_lotka_volterra_trajectory_df['timepoint_years'],
    'prey_data': deterministic_lotka_volterra_trajectory_df['prey_population_sol'],
    'predator_data': deterministic_lotka_volterra_trajectory_df['predator_population_sol']
})

schema = plots.trajectories(distributions, traces=traces)
plots.ipy_display(schema)

## Intervene on uncertain Lotka Volterra

In [19]:
intervened_lotka_volterra = intervene_petri_model(uncertain_lotka_volterra,
                                                  [(25, 'alpha', 1.0)])
intervened_lotka_volterra

LotkaVolterra()

## Sample from intervened model

At $t=25$ we intervene to set $\alpha=1.0$.

In [20]:
intervened_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50
                                  )
intervened_samples

{'alpha': tensor([0.6505, 0.7146, 0.7212, 0.7233, 0.6791, 0.6871, 0.6576, 0.6193, 0.6605,
         0.6691, 0.6597, 0.6402, 0.6904, 0.7292, 0.7001, 0.6128, 0.6335, 0.6042,
         0.6663, 0.6410, 0.6286, 0.6161, 0.6523, 0.7147, 0.6801, 0.6053, 0.6716,
         0.6680, 0.6826, 0.7075, 0.7091, 0.6811, 0.6882, 0.7150, 0.6777, 0.6592,
         0.7314, 0.7156, 0.6983, 0.7169, 0.6834, 0.7064, 0.6792, 0.6254, 0.6535,
         0.6690, 0.6722, 0.6369, 0.6750, 0.7121]),
 'beta': tensor([1.3725, 1.4564, 1.3217, 1.4603, 1.2032, 1.3839, 1.3262, 1.2221, 1.3440,
         1.3108, 1.2440, 1.4025, 1.2329, 1.3744, 1.4309, 1.4027, 1.3040, 1.2550,
         1.2955, 1.3395, 1.4219, 1.1989, 1.3507, 1.4374, 1.3917, 1.3363, 1.2304,
         1.3374, 1.3801, 1.3675, 1.2541, 1.3340, 1.2706, 1.2269, 1.2652, 1.3274,
         1.4002, 1.2284, 1.3687, 1.4202, 1.2669, 1.3323, 1.2601, 1.4331, 1.4013,
         1.3130, 1.2814, 1.2158, 1.2035, 1.4171]),
 'gamma': tensor([1.0368, 0.9453, 1.0539, 0.9756, 1.0483, 0.9569, 0.930

In [38]:
intervened_samples_df = convert_to_output_format(intervened_samples,
                                                 tspan, time_unit="years")
distributions = pd.DataFrame({
    'timepoint_years': prior_samples_df['timepoint_years'], 
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_intervened': intervened_samples_df['prey_population_sol'],
    'predator_intervened': intervened_samples_df['predator_population_sol'],
    })

schema = plots.trajectories(distributions,
                           markers={"Intervention at T=25": 25})
schema = plots.resize(schema, w=700, h=400)
plots.ipy_display(schema)

## Sample from counterfactual distribution

Given that we observe the deterministic trajectory in the factual world,
what would be the trajectory in a counterfactual world where we intervene at $t=25$ to set $\alpha=1.5$?

In [22]:
counterfactual_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50,
                                      inferred_parameters=fitted_parameters
                                  )
counterfactual_samples

{'alpha': tensor([0.6810, 0.6845, 0.6818, 0.6805, 0.6865, 0.6896, 0.6845, 0.6752, 0.6780,
         0.6856, 0.6756, 0.6857, 0.6806, 0.6804, 0.6848, 0.6759, 0.6837, 0.6829,
         0.6836, 0.6834, 0.6841, 0.6835, 0.6834, 0.6781, 0.6815, 0.6875, 0.6809,
         0.6829, 0.6862, 0.6818, 0.6855, 0.6804, 0.6831, 0.6828, 0.6827, 0.6855,
         0.6797, 0.6803, 0.6848, 0.6831, 0.6876, 0.6849, 0.6810, 0.6843, 0.6816,
         0.6849, 0.6847, 0.6846, 0.6837, 0.6798]),
 'beta': tensor([1.2522, 1.2330, 1.2695, 1.2678, 1.2837, 1.3301, 1.2630, 1.2279, 1.2442,
         1.2583, 1.2258, 1.2907, 1.2604, 1.2451, 1.2459, 1.2591, 1.2476, 1.2589,
         1.2961, 1.2687, 1.2594, 1.2694, 1.2383, 1.2362, 1.2816, 1.2857, 1.2677,
         1.2887, 1.2696, 1.2776, 1.2728, 1.2786, 1.2694, 1.2866, 1.2849, 1.2769,
         1.2491, 1.2624, 1.2665, 1.2748, 1.2632, 1.2770, 1.2555, 1.2814, 1.2295,
         1.2600, 1.2852, 1.2540, 1.2383, 1.2287]),
 'gamma': tensor([0.9860, 0.9765, 0.9820, 0.9831, 0.9762, 0.9793, 0.976

In [39]:
counterfactual_samples_df = convert_to_output_format(counterfactual_samples,
                                                     tspan, time_unit="years")
dists = pd.DataFrame({
    'timepoint_years': prior_samples_df['timepoint_years'],    
    'prey_prior': posterior_samples_df['prey_population_sol'],
    'prey_posterior': posterior_samples_df['prey_population_sol'], 
    'predator_posterior': posterior_samples_df['predator_population_sol'],
    'prey_counterfactual': counterfactual_samples_df['prey_population_sol'],
    'predator_counterfactual': counterfactual_samples_df['predator_population_sol']
    })

schema = plots.trajectories(dists, markers={"Intervention at T=25": 25})
plots.ipy_display(schema)